In [1]:
import tensorflow as tf
import pandas as pd
import random
import numpy as np

In [2]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

train_df = pd.read_csv("Train.csv")
dict = {0:"healthy", 1:"fall_armyworm"}
train_df.Label = train_df.Label.map(dict)
print(train_df.head())

               Image_id          Label
0  id_02amazy34fgh2.jpg  fall_armyworm
1  id_02mh3w48pmyc9.jpg        healthy
2  id_02rpb463h9d3w.jpg        healthy
3  id_02wc3jeeao8ol.jpg  fall_armyworm
4  id_03t2hapb8wz8p.jpg  fall_armyworm


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    validation_split = 0.2,
)

train_ds = train_gen.flow_from_dataframe(
    directory = "Images",
    dataframe = train_df,
    x_col = "Image_id",
    y_col = "Label",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training",
)

val_ds = train_gen.flow_from_dataframe(
    directory = "Images",
    dataframe = train_df,
    x_col = "Image_id",
    y_col = "Label",
    target_size = (256,256),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation",
)

Found 1296 validated image filenames belonging to 2 classes.
Found 323 validated image filenames belonging to 2 classes.


In [4]:
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def model(y):
    x = Rescaling(1./255)(y)
    x = Conv2D(64,3,padding="same",activation="relu",strides=(2,2))(x)
    x = MaxPooling2D()(x)
    
    x = Conv2D(128,3,padding="same",activation="relu",strides=(2,2))(x)
    x = Conv2D(128,3,padding="same",activation="relu",strides=(2,2))(x)
    x = Conv2D(256,3,padding="same",activation="relu",strides=(2,2))(x)
    x = MaxPooling2D()(x)

    x = Flatten()(x)
    x = Dense(512,activation="relu")(x)
    x = Dropout(0.2,seed=42)(x)
    x = Dense(512,activation="relu")(x)
    x = Dropout(0.2,seed=42)(x)
    
    output = Dense(2)(x)
    model = Model(y,output)
    return model

In [5]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import CategoricalCrossentropy

model = model(Input(shape=(256,256,3)))
model.summary()
model.compile(RMSprop(learning_rate=1e-3),CategoricalCrossentropy(from_logits=True),metrics=["accuracy"])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 128)       147584    
                                                             

In [6]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

if __name__=="__main__":
    checkpoint = ModelCheckpoint("fall_armyworm.h5",save_weights_only=False,save_best_only=True,monitor="val_accuracy")
    model.fit(train_ds,epochs=20,validation_data=val_ds,callbacks=[checkpoint])
    best = load_model("fall_armyworm.h5")
    loss,acc = best.evaluate(val_ds)
    print("\nAccuracy: {:.2f} %".format(100*acc))
    print("Loss: {:.2f} %".format(100*loss))

Epoch 1/20
41/41 [==============================] - 24s 524ms/step - loss: 0.7614 - accuracy: 0.5208 - val_loss: 0.6914 - val_accuracy: 0.6068
Epoch 2/20
41/41 [==============================] - 21s 510ms/step - loss: 0.7431 - accuracy: 0.5394 - val_loss: 0.6970 - val_accuracy: 0.4768
Epoch 3/20
41/41 [==============================] - 21s 513ms/step - loss: 0.6932 - accuracy: 0.5748 - val_loss: 0.6590 - val_accuracy: 0.6935
Epoch 4/20
41/41 [==============================] - 21s 531ms/step - loss: 0.6628 - accuracy: 0.6628 - val_loss: 0.5059 - val_accuracy: 0.7585
Epoch 5/20
41/41 [==============================] - 21s 523ms/step - loss: 0.6123 - accuracy: 0.7122 - val_loss: 0.4977 - val_accuracy: 0.7492
Epoch 6/20
41/41 [==============================] - 20s 492ms/step - loss: 0.5866 - accuracy: 0.7477 - val_loss: 0.4910 - val_accuracy: 0.7616
Epoch 7/20
41/41 [==============================] - 21s 507ms/step - loss: 0.4748 - accuracy: 0.7778 - val_loss: 0.4317 - val_accuracy: 0.7802